In [53]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [55]:
!cp "/content/drive/MyDrive/colab_data/Capture.zip" /content

In [57]:
!unzip -q "/content/Capture.zip" -d /content/my_dataset

replace /content/my_dataset/bb_1_140613_vehicle_224_55460.mp4_20260114_143204.270.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
import glob
import cv2
import shutil
import pandas as pd


def prepare_yolo_dataset_by_csv(root_path, csv_path, output_path):
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    df = df[['env', 'split']].dropna()
    df = df[df['split'].isin(['train', 'val', 'test'])]

    split_map = dict(zip(df['env'], df['split']))
    print(f"✅ CSV 로드 완료: {len(split_map)}개의 환경(env) 설정 읽음.")

    image_paths = glob.glob(os.path.join(root_path, "**/img/*.png"), recursive=True)
    print(f"총 {len(image_paths)}개의 이미지를 찾았습니다.")

    if len(image_paths) == 0:
        print("❌ 이미지를 찾지 못했습니다. root_path 확인 필요")
        return

    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(output_path, split, 'labels'), exist_ok=True)

    processed_count = 0
    missing_env_count = 0

    for img_path in image_paths:
        folder_name = img_path.split(os.sep)[-3]
        file_name = os.path.basename(img_path)

        target_split = split_map.get(folder_name)
        if not target_split:
            missing_env_count += 1
            continue

        new_base = f"{folder_name}_{file_name}"
        new_label = new_base.replace(".png", ".txt")

        org_label_path = img_path.replace("img", "new_txt").replace(".png", ".txt")
        if not os.path.exists(org_label_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue
        h, w, _ = img.shape

        yolo_labels = []
        with open(org_label_path) as f:
            for line in f:
                data = list(map(float, line.split()))
                if len(data) < 5:
                    continue

                x1, y1, x2, y2 = data[:4]
                class_id = int(data[4])

                xc = ((x1 + x2) / 2) / w
                yc = ((y1 + y2) / 2) / h
                nw = (x2 - x1) / w
                nh = (y2 - y1) / h

                yolo_labels.append(
                    f"{class_id} {xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}"
                )

        if yolo_labels:
            shutil.copy(
                img_path,
                os.path.join(output_path, target_split, "images", new_base)
            )
            with open(
                os.path.join(output_path, target_split, "labels", new_label), "w"
            ) as f:
                f.write("\n".join(yolo_labels))

            processed_count += 1

    print("-" * 30)
    print("✨ 처리 완료!")
    print(f"- 성공적으로 변환된 이미지: {processed_count}장")
    print(f"- CSV에 정의되지 않아 제외된 폴더: {missing_env_count}개")
    print(f"- 결과 저장소: {os.path.abspath(output_path)}")


In [ ]:
prepare_yolo_dataset_by_csv(
    root_path="/content/my_dataset",
    csv_path="/content/drive/MyDrive/colab_data/AI_Project/split_assignment.csv",
    output_path="/content/my_car_dataset"
)

✅ CSV 로드 완료: 165개의 환경(env) 설정 읽음.
총 33873개의 이미지를 찾았습니다.
------------------------------
✨ 처리 완료!
- 성공적으로 변환된 이미지: 33873장
- CSV에 정의되지 않아 제외된 폴더: 0개
- 결과 저장소: /content/my_car_dataset


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.1 MB/s eta 0:00:00


In [ ]:
#2-2) 모델 정의 (ResNet50-FPN + ROIAlign + shared head + 2 heads)

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.faster_rcnn import TwoMLPHead
from torchvision.models.detection.transform import GeneralizedRCNNTransform

def build_backbone_resnet50_fpn():
    # torchvision 버전 호환 처리
    try:
        from torchvision.models import ResNet50_Weights
        return resnet_fpn_backbone("resnet50", weights=ResNet50_Weights.DEFAULT)
    except Exception:
        # 구버전에서는 pretrained 인자를 받을 수 있음
        return resnet_fpn_backbone("resnet50", pretrained=True)

class ROIClassifier(nn.Module):
    def __init__(
        self,
        num_location=5,
        num_action=4,
        pos_weight=None,                 # torch.Tensor[4]
        min_size=640, max_size=1024,      # transform resize
        roi_output_size=7,
        roi_sampling_ratio=2,
        representation_size=1024,
        image_mean=(0.485,0.456,0.406),
        image_std=(0.229,0.224,0.225),
    ):
        super().__init__()

        self.transform = GeneralizedRCNNTransform(
            min_size=min_size,
            max_size=max_size,
            image_mean=list(image_mean),
            image_std=list(image_std),
        )

        self.backbone = build_backbone_resnet50_fpn()
        out_channels = self.backbone.out_channels  # usually 256

        self.roi_pooler = MultiScaleRoIAlign(
            featmap_names=["0","1","2","3"],
            output_size=roi_output_size,
            sampling_ratio=roi_sampling_ratio,
        )

        self.box_head = TwoMLPHead(
            in_channels=out_channels * roi_output_size * roi_output_size,
            representation_size=representation_size,
        )

        self.location_head = nn.Linear(representation_size, num_location)
        self.action_head   = nn.Linear(representation_size, num_action)

        # action loss용 pos_weight 저장
        if pos_weight is not None:
            self.register_buffer("pos_weight", pos_weight.clone().detach())
        else:
            self.pos_weight = None

    def forward(self, images, targets=None):
        """
        images: list[Tensor(3,H,W)]
        targets: list[dict] with keys:
          boxes: FloatTensor[N,4] xyxy (원본 픽셀 좌표계)
          location: LongTensor[N]
          action: FloatTensor[N,4] (0/1)
        """
        if self.training and targets is None:
            raise ValueError("Training requires targets")

        image_list, targets = self.transform(images, targets)
        features = self.backbone(image_list.tensors)  # dict[str,Tensor]

        boxes = [t["boxes"] for t in targets]
        pooled = self.roi_pooler(features, boxes, image_list.image_sizes)  # [sumN, C, S, S]
        pooled = pooled.flatten(start_dim=1)                               # [sumN, C*S*S]
        rep = self.box_head(pooled)                                        # [sumN, repr]

        loc_logits = self.location_head(rep)  # [sumN, 5]
        act_logits = self.action_head(rep)    # [sumN, 4]

        if self.training:
            loc_t = torch.cat([t["location"] for t in targets], dim=0)  # [sumN]
            act_t = torch.cat([t["action"] for t in targets], dim=0)    # [sumN,4]

            loss_loc = F.cross_entropy(loc_logits, loc_t)

            if self.pos_weight is None:
                loss_act = F.binary_cross_entropy_with_logits(act_logits, act_t)
            else:
                loss_act = F.binary_cross_entropy_with_logits(
                    act_logits, act_t, pos_weight=self.pos_weight
                )

            return {"loss_location": loss_loc, "loss_action": loss_act}

        # inference
        return F.softmax(loc_logits, dim=1), torch.sigmoid(act_logits)


In [59]:
import glob
import os
import cv2
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image as PILImage
import torchvision.transforms.functional as TF

# --- 1. 설정 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_root = '/content/my_dataset'
output_root = '/content/SafeDrive_T4_HighPerf/test_sequences_combined'
os.makedirs(output_root, exist_ok=True)

# 성민님 모델 가중치 경로 ★
TEAMMATE_WEIGHTS_PATH = "/content/drive/MyDrive/colab_data/roi.pt"

# --- 2. 모델 로드 ---
# (1) YOLO 모델 로드
if 'model' in globals():
    yolo_model = model
    print("✅ YOLO 모델(메모리) 사용")
else:
    from ultralytics import YOLO
    yolo_path = '/content/drive/MyDrive/colab_data/yolov8nv1_best.pt'
    if os.path.exists(yolo_path):
        yolo_model = YOLO(yolo_path)
        print("✅ YOLO 모델 새로 로드 완료")
    else:
        raise FileNotFoundError("❌ YOLO 모델을 찾을 수 없습니다.")

# (2) ROI 모델(팀원 모델) 로드
print(f"📂 팀원 모델 로드 시도: {TEAMMATE_WEIGHTS_PATH}")
if not os.path.exists(TEAMMATE_WEIGHTS_PATH):
    raise FileNotFoundError("❌ 팀원 모델 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

try:
    # 모델 구조 생성 (ROIClassifier 클래스가 메모리에 있어야 함)
    roi_model = ROIClassifier(num_location=5, num_action=4).to(device)

    # 가중치 파일 로드
    ckpt = torch.load(TEAMMATE_WEIGHTS_PATH, map_location=device, weights_only=False)

    # state_dict 키 처리
    if isinstance(ckpt, dict) and "model_state" in ckpt:
        state_dict = ckpt["model_state"]
    else:
        state_dict = ckpt

    # 로드 (strict=False로 설정하여 pos_weight 등 불일치 무시)
    roi_model.load_state_dict(state_dict, strict=False)
    roi_model.eval()
    print("✅ 팀원 모델(ROI) 로드 성공!")
except Exception as e:
    print(f"❌ 모델 로드 중 오류 발생: {e}")
    raise e

# --- 3. 시퀀스 그룹화 ---
all_test_images = glob.glob(os.path.join(dataset_root,  'bb*.png'))
sequences = {}

for img_path in all_test_images:
    filename = os.path.basename(img_path)
    try:
        # 파일명에서 시퀀스 이름 추출
        seq_name = filename.rsplit('_', 1)[0]
        if seq_name not in sequences:
            sequences[seq_name] = []
        sequences[seq_name].append(img_path)
    except IndexError:
        pass

print(f"📂 총 {len(sequences)}개의 시퀀스를 발견했습니다. 통합 분석을 시작합니다...")

# --- 4. 내부 파이프라인 함수 ---
@torch.no_grad()
def process_single_frame(image_path, yolo_model, roi_model, device):
    img_cv = cv2.imread(image_path)
    if img_cv is None: return None, []

    filename = os.path.basename(image_path)

    # YOLO Input
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    img_pil = PILImage.fromarray(img_rgb)

    # YOLO Inference
    yolo_results = yolo_model(img_rgb, verbose=False)
    boxes = yolo_results[0].boxes.xyxy.cpu()
    clss = yolo_results[0].boxes.cls.cpu().int()
    names = yolo_model.names

    if len(boxes) == 0: return img_cv, []

    # ROI Input
    img_tensor = TF.to_tensor(img_pil).to(device)
    target_dict = [{"boxes": boxes.to(device)}]

    # ROI Inference
    loc_probs, act_probs = roi_model([img_tensor], target_dict)
    loc_preds = loc_probs.argmax(dim=1).cpu().numpy()
    act_probs = act_probs.cpu().numpy()

    # Config
    LOC_NAMES = ["My", "Next", "Opposite", "RoadSide", "None"]
    ACT_NAMES = ["Brake", "Left", "Right", "Hazard"]

    frame_detections = []
    output_img = img_cv.copy()

    for i, box in enumerate(boxes.numpy()):
        x1, y1, x2, y2 = map(int, box)
        cls_id = int(clss[i])
        class_name = names[cls_id] if names and cls_id in names else str(cls_id)

        loc_idx = int(loc_preds[i])
        loc_text = LOC_NAMES[loc_idx] if loc_idx < len(LOC_NAMES) else str(loc_idx)

        # Actions
        act_vals = act_probs[i]
        is_active = (act_vals > 0.5).astype(int)
        actions_str = [ACT_NAMES[j] for j, active in enumerate(is_active) if active]
        act_text = ",".join(actions_str) if actions_str else "Normal"

        # Append Data
        frame_detections.append({
            "Filename": filename,
            "Class_Name": class_name,
            "Location_Name": loc_text,
            "Action_String": act_text,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "Class_ID": cls_id,
            "Location_ID": loc_idx,
            "Brake": is_active[0], "Left": is_active[1], "Right": is_active[2], "Hazard": is_active[3]
        })

        # Draw
        label = f"[{class_name}] {loc_text}|{act_text}"
        cv2.rectangle(output_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        (w_t, h_t), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(output_img, (x1, y1 - 20), (x1 + w_t, y1), (0, 255, 0), -1)
        cv2.putText(output_img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    return output_img, frame_detections

# --- 5. 전체 시퀀스 실행 ---
for seq_name, img_list in tqdm(sequences.items(), desc="Total Progress"):
    sorted_images = sorted(img_list)
    if not sorted_images: continue

    # 저장 경로
    seq_save_dir = os.path.join(output_root, seq_name)
    os.makedirs(seq_save_dir, exist_ok=True)

    video_path = os.path.join(seq_save_dir, f"{seq_name}_combined.mp4")
    csv_path = os.path.join(seq_save_dir, f"{seq_name}_combined.csv")

    # Video Init
    sample = cv2.imread(sorted_images[0])
    h, w = sample.shape[:2]
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

    seq_detections = []

    for img_path in sorted_images:
        res_img, dets = process_single_frame(img_path, yolo_model, roi_model, device)
        if res_img is not None:
            out.write(res_img)
            seq_detections.extend(dets)

    out.release()

    if seq_detections:
        df = pd.DataFrame(seq_detections)
        cols = ["Filename", "x1", "y1", "x2", "y2", "Class_ID", "Location_ID", "Brake", "Left", "Right", "Hazard", "Class_Name", "Location_Name", "Action_String"]
        exist_cols = [c for c in cols if c in df.columns]
        df[exist_cols].to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n✨ 모든 통합 분석 완료! 결과 경로: {output_root}")

✅ YOLO 모델 새로 로드 완료
📂 팀원 모델 로드 시도: /content/drive/MyDrive/colab_data/roi.pt
✅ 팀원 모델(ROI) 로드 성공!
📂 총 2개의 시퀀스를 발견했습니다. 통합 분석을 시작합니다...


Total Progress: 100%|██████████| 2/2 [00:06<00:00,  3.41s/it]


✨ 모든 통합 분석 완료! 결과 경로: /content/SafeDrive_T4_HighPerf/test_sequences_combined


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
# --- Install ---
!pip install numpy filterpy scipy imageio tqdm
!git clone https://github.com/abewley/sort.git

Cloning into 'sort'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 208 (delta 45), reused 40 (delta 40), pack-reused 159 (from 1)
Receiving objects: 100% (208/208), 1.20 MiB | 5.90 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [35]:
%%writefile /content/sort_tracker.py
# =====================================================
# SORT: Simple Online and Realtime Tracking
# Source adapted from https://github.com/abewley/sort
# =====================================================

import numpy as np
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


def iou(bb_test, bb_gt):
    xx1 = np.maximum(bb_test[0], bb_gt[0])
    yy1 = np.maximum(bb_test[1], bb_gt[1])
    xx2 = np.minimum(bb_test[2], bb_gt[2])
    yy2 = np.minimum(bb_test[3], bb_gt[3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1]) +
        (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh
    )
    return o


class KalmanBoxTracker:
    count = 0

    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array(
            [[1,0,0,0,1,0,0],
            [0,1,0,0,0,1,0],
            [0,0,1,0,0,0,1],
            [0,0,0,1,0,0,0],
            [0,0,0,0,1,0,0],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]]
        )
        self.kf.H = np.array([
            [1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [0,0,1,0,0,0,0],
            [0,0,0,1,0,0,0]
        ])
        self.kf.R[2:,2:] *= 10.
        self.kf.P[4:,4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01
        self.kf.Q[4:,4:] *= 0.01

        # Fix: only take first 4 elements (coords)
        self.kf.x[:4] = bbox[:4].reshape((4,1))
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1
        self.history = []
        self.hits = 0
        self.hit_streak = 0
        self.age = 0

    def update(self, bbox):
        self.time_since_update = 0
        self.history = []
        self.hits += 1
        self.hit_streak += 1
        # Fix: only take first 4 elements (coords)
        self.kf.update(bbox[:4].reshape((4,1)))

    def predict(self):
        if (self.kf.x[6] + self.kf.x[2]) <= 0:
            self.kf.x[6] *= 0.0
        self.kf.predict()
        self.age += 1
        if self.time_since_update > 0:
            self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.kf.x)
        return self.kf.x


class Sort:
    def __init__(self, max_age=5, min_hits=2, iou_threshold=0.3):
        self.max_age = max_age
        self.min_hits = min_hits
        self.iou_threshold = iou_threshold
        self.trackers = []
        self.frame_count = 0

    def update(self, dets=np.empty((0,4))):
        self.frame_count += 1

        trks = np.zeros((len(self.trackers), 4))
        to_del = []
        for t, trk in enumerate(self.trackers):
            pos = trk.predict()
            trks[t] = pos[:4].reshape((1,4))
            if np.any(np.isnan(pos)):
                to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(
            dets, trks, self.iou_threshold
        )

        for m in matched:
            self.trackers[m[1]].update(dets[m[0]])

        for i in unmatched_dets:
            self.trackers.append(KalmanBoxTracker(dets[i]))

        ret = []
        for trk in self.trackers:
            if trk.time_since_update < 1 and (trk.hits >= self.min_hits or self.frame_count <= self.min_hits):
                ret.append(np.concatenate((trk.kf.x[:4].reshape((1,4)), [[trk.id]]), axis=1))
        return np.concatenate(ret) if len(ret) > 0 else np.empty((0,5))


def associate_detections_to_trackers(detections, trackers, iou_threshold):
    if len(trackers) == 0:
        return np.empty((0,2), dtype=int), np.arange(len(detections)), np.empty((0), dtype=int)

    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d,t] = iou(det, trk)

    matched_indices = linear_sum_assignment(-iou_matrix)
    matched_indices = np.array(list(zip(*matched_indices)))

    # Fix for IndexError when matched_indices is empty
    if len(matched_indices) == 0:
        matched_indices = np.empty((0, 2), dtype=int)

    unmatched_detections = []
    for d in range(len(detections)):
        if d not in matched_indices[:,0]:
            unmatched_detections.append(d)

    unmatched_trackers = []
    for t in range(len(trackers)):
        if t not in matched_indices[:,1]:
            unmatched_trackers.append(t)

    matches = []
    for m in matched_indices:
        if iou_matrix[m[0], m[1]] < iou_threshold:
            unmatched_detections.append(m[0])
            unmatched_trackers.append(m[1])
        else:
            matches.append(m.reshape(1,2))

    if len(matches) == 0:
        matches = np.empty((0,2), dtype=int)
    else:
        matches = np.concatenate(matches, axis=0)

    return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


Overwriting /content/sort_tracker.py


In [36]:
import sys
sys.path.insert(0, "/content")

from sort_tracker import Sort

tracker = Sort()
print("SORT import OK")

SORT import OK


In [37]:
import cv2
import os
import glob
import numpy as np
import imageio
from collections import deque
from tqdm import tqdm

In [38]:
class IntentAwareRiskManager:
    def __init__(self, window_size=5):
        self.window_size = window_size
        self.history = {}  # {track_id: deque[(cx, cy)]}

    def _update_history(self, track_id, center):
        if track_id not in self.history:
            self.history[track_id] = deque(maxlen=self.window_size)
        self.history[track_id].append(center)

    def _is_long_static(self, centers, move_thresh=3.0, min_static_frames=6):
        if len(centers) < min_static_frames + 1:
            return False

        diffs = [
            np.linalg.norm(np.array(centers[i]) - np.array(centers[i-1]))
            for i in range(1, len(centers))
        ]
        return all(d < move_thresh for d in diffs[-min_static_frames:])

    def _compute_2d_traj_metrics(self, centers, fps, img_w, img_h, eps=1e-3):
        """
        return: ttc, v_rel, d_curr
        """
        if len(centers) < 2:
            return float('inf'), 0.0, float('inf')

        ego = np.array([img_w / 2, img_h])
        p_prev = np.array(centers[-2])
        p_curr = np.array(centers[-1])

        d_prev = np.linalg.norm(p_prev - ego)
        d_curr = np.linalg.norm(p_curr - ego)
        v_rel = (d_prev - d_curr) * fps  # px/sec

        # 접근 안 하면 무효
        if v_rel < 5.0:
            return float('inf'), v_rel, d_curr

        # 장기 정지 필터
        if self._is_long_static(centers):
            return float('inf'), v_rel, d_curr

        ttc = d_curr / max(v_rel, eps)
        return ttc, v_rel, d_curr

    def calculate_risk(self, track_id, bbox, location, actions, fps, img_w, img_h):
        cx = (bbox[0] + bbox[2]) / 2
        cy = (bbox[1] + bbox[3]) / 2
        self._update_history(track_id, (cx, cy))
        centers = self.history[track_id]

        # --- Step 1: Physical risk ---
        ttc, v_rel, d_curr = self._compute_2d_traj_metrics(
            centers, fps, img_w, img_h
        )

        base_score = 0.0
        if ttc < 5.0:
            base_score = min(50.0, 50.0 * (3.0 / max(ttc, 0.5)))

        # --- Step 2: Semantic reasoning ---
        semantic_bonus = 0.0
        omega = 1.0
        reason = "Normal"

        # Ignore irrelevant zones
        if location in [2, 3, 4]:
            return {
                "score": 0.0,
                "ttc": "Safe",
                "reason": "Ignored",
                "level": "SAFE"
            }

        HIGH_SPEED = 80.0  # px/sec

        # Ego lane
        if location == 0:
            if ttc < 2.0 and v_rel > HIGH_SPEED:
                reason = "Rapid Approach"
                semantic_bonus = 10.0
            elif ttc < 2.0:
                reason = "Close Following"
            else:
                reason = "Normal Deceleration"

        # Adjacent lane (cut-in)
        elif location == 1:
            if actions[1] == 1 or actions[2] == 1:
                reason = "Dangerous Cut-in"
                semantic_bonus = 10.0
                omega = 1.3
            else:
                reason = "Approaching"
                omega = 0.8

        final_risk = min(base_score * omega + semantic_bonus, 100.0)

        return {
            "score": round(final_risk, 2),
            "ttc": round(ttc, 2) if ttc != float('inf') else "Safe",
            "reason": reason,
            "level": self._get_risk_level(final_risk),
        }

    def _get_risk_level(self, score):
        if score > 50: return "DANGER"
        if score > 20: return "WARNING"
        return "SAFE"


In [39]:
import cv2
import numpy as np
import torch
import os
from tqdm import tqdm
from PIL import Image as PILImage
import torchvision.transforms.functional as TF

# --- RELOAD SORT TO APPLY FIX ---
import importlib
import sort_tracker
importlib.reload(sort_tracker)
from sort_tracker import Sort

# Ensure dependencies are loaded
if 'IntentAwareRiskManager' not in globals():
    # Re-define or ensure cell is run. Assuming it is run based on context.
    pass

# --- Color Palette for Risk Levels ---
RISK_COLORS = {
    "SAFE": (0, 255, 0),       # Green
    "WARNING": (0, 165, 255),  # Orange
    "DANGER": (0, 0, 255)      # Red
}

def process_sequence_with_risk(seq_name, image_paths, output_dir, yolo_model, roi_model, device):
    os.makedirs(output_dir, exist_ok=True)
    video_path = os.path.join(output_dir, f"{seq_name}_risk_analysis.mp4")

    # Sort images to ensure temporal order
    image_paths = sorted(image_paths)
    if not image_paths: return

    # Init Tracker & Risk Manager per sequence
    # min_hits=1: 감지 즉시 트래킹 시작 (반응 속도 최우선)
    tracker = Sort(max_age=5, min_hits=1, iou_threshold=0.3)
    risk_manager = IntentAwareRiskManager(window_size=10)

    # Video Writer Init
    sample_img = cv2.imread(image_paths[0])
    h, w = sample_img.shape[:2]
    fps = 10.0 # Assumption for playback speed
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    print(f"🎬 Processing {seq_name} ({len(image_paths)} frames)...")

    # Use torch.no_grad() for inference to save memory and avoid grad errors
    with torch.no_grad():
        for img_path in tqdm(image_paths, leave=False):
            img_cv = cv2.imread(img_path)
            if img_cv is None: continue

            # [Safety Fix] Resize frame if dimensions mismatch
            if img_cv.shape[:2] != (h, w):
                img_cv = cv2.resize(img_cv, (w, h))

            # 1. YOLO Detection
            img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
            results = yolo_model(img_rgb, verbose=False)

            dets_to_sort = np.empty((0, 5))
            if results[0].boxes.id is not None:
                 # If YOLO has tracking, ignore it, we use SORT. Just take boxes & conf
                 pass

            # Extract boxes for SORT [x1, y1, x2, y2, score]
            # We use confidence as score
            boxes = results[0].boxes.xyxy.cpu().numpy()
            confs = results[0].boxes.conf.cpu().numpy()
            if len(boxes) > 0:
                dets_to_sort = np.column_stack((boxes, confs))

            # 2. SORT Tracking
            # track_dets: [x1, y1, x2, y2, track_id]
            track_dets = tracker.update(dets_to_sort)

            # 3. ROI Classification & Risk Calculation
            if len(track_dets) > 0:
                # Prepare inputs for ROI model (using tracked boxes)
                # ROI model expects boxes in a list of dicts
                roi_boxes_tensor = torch.tensor(track_dets[:, :4]).float().to(device)
                img_tensor = TF.to_tensor(PILImage.fromarray(img_rgb)).to(device)

                # ROI Forward
                # Note: We process all tracked objects in batch
                roi_loc_probs, roi_act_probs = roi_model([img_tensor], [{"boxes": roi_boxes_tensor}])

                # Use .detach() just in case, though torch.no_grad() handles most cases
                loc_preds = roi_loc_probs.argmax(dim=1).detach().cpu().numpy()
                act_probs = roi_act_probs.detach().cpu().numpy()

                LOC_NAMES = ["My", "Next", "Opposite", "RoadSide", "None"]
                ACT_NAMES = ["Brake", "Left", "Right", "Hazard"]

                for i, trk in enumerate(track_dets):
                    x1, y1, x2, y2, track_id = map(int, trk)

                    # Get ROI results
                    loc_idx = loc_preds[i]
                    act_vals = act_probs[i]

                    # Decode ROI
                    loc_name = LOC_NAMES[loc_idx] if loc_idx < len(LOC_NAMES) else "Unknown"
                    is_active = (act_vals > 0.5).astype(int)

                    # 4. Risk Calculation
                    risk_result = risk_manager.calculate_risk(
                        track_id=track_id,
                        bbox=[x1, y1, x2, y2],
                        location=loc_idx,
                        actions=is_active,
                        fps=fps,
                        img_w=w,
                        img_h=h
                    )

                    # 5. Visualization
                    risk_level = risk_result["level"]
                    risk_score = risk_result["score"]
                    reason = risk_result["reason"]
                    color = RISK_COLORS.get(risk_level, (0, 255, 0))

                    # Bounding Box
                    cv2.rectangle(img_cv, (x1, y1), (x2, y2), color, 2)

                    # Info Box
                    info_text = f"ID:{track_id} | {loc_name}"
                    risk_text = f"{risk_level} ({risk_score})"
                    reason_text = f"{reason}"

                    # Draw labels
                    cv2.putText(img_cv, info_text, (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(img_cv, risk_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    if risk_level != "SAFE":
                         cv2.putText(img_cv, reason_text, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            out.write(img_cv)

    out.release()

# --- Run on All Sequences ---
OUTPUT_ROOT_RISK = "/content/SafeDrive_T4_HighPerf/integrated_risk_videos"

# 'sequences' variable should be available from previous cells
# If not, we regenerate it briefly:
if 'sequences' not in globals() or not sequences:
    print("⚠️ sequences not found, rescanning...")
    test_imgs = glob.glob(os.path.join('/content/my_car_dataset', 'test', 'images', '*.png'))
    sequences = {}
    for p in test_imgs:
        s_name = os.path.basename(p).rsplit('_', 1)[0]
        sequences.setdefault(s_name, []).append(p)

print(f"🚀 Starting Risk Analysis on {len(sequences)} sequences...")

for seq_name, img_list in sequences.items():
    process_sequence_with_risk(
        seq_name,
        img_list,
        OUTPUT_ROOT_RISK,
        yolo_model,
        roi_model,
        device
    )

print(f"\n✅ Done! Check results in: {OUTPUT_ROOT_RISK}")

🚀 Starting Risk Analysis on 1 sequences...
🎬 Processing bb_1_140613_vehicle_224_55460.mp4_20260114 (29 frames)...



✅ Done! Check results in: /content/SafeDrive_T4_HighPerf/integrated_risk_videos


In [60]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
from collections import deque
from sort_tracker import Sort

# --- 설정 및 경로 ---
INPUT_ROOT = '/content/SafeDrive_T4_HighPerf/test_sequences_combined'
IMAGE_ROOT = '/content/my_dataset'
OUTPUT_ROOT = '/content/SafeDrive_T4_HighPerf/final_risk_result_v4'
os.makedirs(OUTPUT_ROOT, exist_ok=True)

FOCAL_LENGTH = 800
ACTUAL_WIDTH = 1.8
FPS = 10.0

# --- IoU 계산 ---
def get_iou(bb1, bb2):
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])
    if x_right < x_left or y_bottom < y_top: return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
    return intersection_area / float(bb1_area + bb2_area - intersection_area)

# --- 위험도 판단 ---
def calculate_risk_advanced(dist, ttc, loc, lights, is_static):
    base_score = 0
    reasons = []

    if is_static: return 0, ["Static(Safe)"]
    if loc >= 4: return 0, ["Ignored"]

    # 물리적 위험
    if dist <= 5.0: base_score, reasons = 80, ["Crit.Dist"]
    elif ttc < 2.0: base_score, reasons = 75, [f"Crit.TTC({ttc:.1f}s)"]
    elif dist <= 10.0: base_score, reasons = 50, ["Close"]
    elif ttc < 4.0: base_score, reasons = 40, [f"Appr({ttc:.1f}s)"]
    elif dist <= 20.0: base_score, reasons = 20, ["Near"]
    final_score = base_score

    # 브레이크
    if lights['Brake'] == 1:
        if loc == 0: final_score += 30; reasons.append("Brake")
        elif loc == 1: final_score += 15; reasons.append("SideBrake")

    # 비상등
    if lights['Hazard'] == 1: final_score += 20; reasons.append("Hazard")

    # 끼어들기
    if (lights['Left'] == 1 or lights['Right'] == 1) and loc == 1:
        if dist < 15.0 or ttc < 5.0: final_score += 15; reasons.append("Cut-in")
        else: final_score += 5

    return min(final_score, 100), reasons

# --- 이미지 인덱싱 ---
def index_images(root_dir):
    print(f"🔍 이미지 경로 인덱싱 중... ({root_dir})")
    image_map = {}
    count = 0
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_map[file] = os.path.join(root, file)
                count += 1
    print(f"✅ 총 {count}개의 이미지를 찾았습니다.")
    return image_map

# --- 메인 처리 ---
def process_combined_sequences():
    # 0. 이미지 인덱싱
    image_map = index_images(IMAGE_ROOT)

    # 1. 시퀀스 CSV 찾기
    # 폴더 구조: INPUT_ROOT / seq_name / seq_name_combined.csv
    # 혹은 파일이 바로 있을 수도 있으므로 재귀 탐색
    csv_files = glob.glob(os.path.join(INPUT_ROOT, '**', '*combined.csv'), recursive=True)
    print(f"📂 총 {len(csv_files)}개의 통합 CSV 파일을 발견했습니다.")

    for csv_path in csv_files:
        try:
            seq_name = os.path.basename(os.path.dirname(csv_path))
            # 만약 폴더명이 없으면 파일명에서 추출
            if not seq_name or seq_name == os.path.basename(INPUT_ROOT):
                 seq_name = os.path.basename(csv_path).replace('_combined.csv', '')

            print(f"🚀 Processing Sequence: {seq_name}")

            df = pd.read_csv(csv_path)
            if df.empty:
                print("⚠️ CSV가 비어있습니다. 건너뜁니다.")
                continue

            df = df.sort_values(by='Filename')
            unique_files = df['Filename'].unique()

            if len(unique_files) == 0:
                print("⚠️ 처리할 프레임이 없습니다.")
                continue

            # 저장 경로 설정
            save_video_path = os.path.join(OUTPUT_ROOT, f"{seq_name}_risk_final.mp4")
            save_frames_dir = os.path.join(OUTPUT_ROOT, f"{seq_name}_frames")
            os.makedirs(save_frames_dir, exist_ok=True)

            # 첫 프레임 확인 및 VideoWriter 초기화
            first_file = unique_files[0]
            if first_file not in image_map:
                print(f"❌ 첫 프레임({first_file})을 찾을 수 없습니다. 경로를 확인해주세요.")
                # 혹시 모르니 경로 직접 확인
                continue

            sample_img = cv2.imread(image_map[first_file])
            if sample_img is None:
                print(f"❌ 이미지 읽기 실패: {image_map[first_file]}")
                continue

            h, w = sample_img.shape[:2]
            out = cv2.VideoWriter(save_video_path, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (w, h))

            tracker = Sort(max_age=5, min_hits=3, iou_threshold=0.3)
            dist_history = {}

            processed_frames = 0
            for filename in tqdm(unique_files, desc=f"Seq: {seq_name}", leave=False):
                if filename not in image_map:
                    continue

                img_path = image_map[filename]
                frame = cv2.imread(img_path)
                if frame is None: continue

                frame_data = df[df['Filename'] == filename]

                # SORT 입력
                dets_for_sort = []
                original_attrs = []
                for _, row in frame_data.iterrows():
                    dets_for_sort.append([row['x1'], row['y1'], row['x2'], row['y2'], 1.0])
                    original_attrs.append(row)

                dets_for_sort = np.array(dets_for_sort)
                if len(dets_for_sort) == 0:
                    dets_for_sort = np.empty((0, 5))

                tracked_objs = tracker.update(dets_for_sort)

                for trk in tracked_objs:
                    tx1, ty1, tx2, ty2, trk_id = map(int, trk[:5])
                    trk_box = [tx1, ty1, tx2, ty2]

                    # 매칭
                    best_iou, matched_row = 0, None
                    for row in original_attrs:
                        org_box = [row['x1'], row['y1'], row['x2'], row['y2']]
                        iou = get_iou(trk_box, org_box)
                        if iou > best_iou:
                            best_iou, matched_row = iou, row

                    if matched_row is None or best_iou < 0.3: continue

                    # 거리 & TTC
                    w_pixel = tx2 - tx1
                    current_dist = (FOCAL_LENGTH * ACTUAL_WIDTH) / w_pixel if w_pixel > 0 else 0

                    if trk_id not in dist_history: dist_history[trk_id] = deque(maxlen=5)
                    dist_history[trk_id].append(current_dist)

                    is_static = False
                    recent = dist_history[trk_id]
                    if len(recent) >= 3 and all(abs(recent[i]-recent[i-1]) < 0.5 for i in range(1, len(recent))):
                        is_static = True

                    ttc = float('inf')
                    if len(recent) >= 2:
                        v_rel = (recent[-2] - current_dist) * FPS
                        if v_rel > 0.1: ttc = current_dist / v_rel

                    # 위험도
                    lights = {k: matched_row[k] for k in ['Brake', 'Left', 'Right', 'Hazard']}
                    score, reasons = calculate_risk_advanced(
                        current_dist, ttc, int(matched_row['Location_ID']), lights, is_static
                    )

                    # 시각화
                    color = (0, 0, 255) if score >= 80 else (0, 165, 255) if score >= 50 else (0, 255, 0)
                    cv2.rectangle(frame, (tx1, ty1), (tx2, ty2), color, 2)

                    info = f"ID:{trk_id} [{matched_row['Class_Name']}] {matched_row['Location_Name']}"
                    risk = f"{'DANGER' if score>=80 else 'WARN' if score>=50 else 'Safe'} ({score})"
                    dist_info = f"{current_dist:.1f}m | TTC:{ttc:.1f}s" if ttc != float('inf') else f"{current_dist:.1f}m | TTC:Inf"

                    cv2.putText(frame, info, (tx1, ty1-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    cv2.putText(frame, risk, (tx1, ty1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                    cv2.putText(frame, dist_info, (tx1, ty2+15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 1)
                    if reasons and score > 0:
                        cv2.putText(frame, ",".join(reasons), (tx1, ty2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

                out.write(frame)
                cv2.imwrite(os.path.join(save_frames_dir, filename), frame)
                processed_frames += 1

            out.release()
            print(f"   ✅ 완료: {processed_frames}장 처리됨 -> {save_video_path}")

        except Exception as e:
            print(f"❌ 오류 발생 ({seq_name}): {e}")
            import traceback
            traceback.print_exc()

    print(f"\n✨ 모든 처리 완료! 결과 확인: {OUTPUT_ROOT}")

process_combined_sequences()

🔍 이미지 경로 인덱싱 중... (/content/my_dataset)
✅ 총 135540개의 이미지를 찾았습니다.
📂 총 18개의 통합 CSV 파일을 발견했습니다.
🚀 Processing Sequence: ETRINear_Fog
❌ 첫 프레임(ETRINear_Fog_00000797.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: bb_1_140613_vehicle_224_55460.mp4_20260114


   ✅ 완료: 29장 처리됨 -> /content/SafeDrive_T4_HighPerf/final_risk_result_v4/bb_1_140613_vehicle_224_55460.mp4_20260114_risk_final.mp4
🚀 Processing Sequence: bb_1_210701_vehicle_217_21650.mp4
❌ 첫 프레임(bb_1_210701_vehicle_217_21650.mp4_12.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: TailLight04
❌ 첫 프레임(TailLight04_000044.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: bb_1_211026_vehicle_144_301.mp4


   ✅ 완료: 19장 처리됨 -> /content/SafeDrive_T4_HighPerf/final_risk_result_v4/bb_1_211026_vehicle_144_301.mp4_risk_final.mp4
🚀 Processing Sequence: TailLight57
❌ 첫 프레임(TailLight57_000020.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: bb_1_150402_vehicle_142_073.mp4
❌ 첫 프레임(bb_1_150402_vehicle_142_073.mp4_0.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: OisamtoBanseok_Snow
❌ 첫 프레임(OisamtoBanseok_Snow_00021609.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: RAIN_Noeun
❌ 첫 프레임(RAIN_Noeun_000090.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: CounterClock2_Fog
❌ 첫 프레임(CounterClock2_Fog_00000368.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: bb_1_200714_vehicle_200_28840.mp4
❌ 첫 프레임(bb_1_200714_vehicle_200_28840.mp4_10.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: Yeonhwi
❌ 첫 프레임(Yeonhwi_000009.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: TailLight27
❌ 첫 프레임(TailLight27_000036.png)을 찾을 수 없습니다. 경로를 확인해주세요.
🚀 Processing Sequence: bb_1_160612_vehicle_212_53256.mp4